In [42]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
from selenium import webdriver
import time 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from urllib.parse import urljoin
import csv


In [43]:
def get_product_urls():
    #create an array for product urls       
    product_urls = []

    #iterate through each url in list of domain urls
    for i in range(30, 60):
        url = f"https://www.mercari.com/us/category/{i}/"
        driver = webdriver.Chrome()
        driver.get(url)
        driver.set_page_load_timeout(12)

        screen_height = driver.execute_script("return window.screen.height;")
        i = 1

        #scroll to the bottom of page for given url 
        
        while True: 
            
            try:
                driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))

                i+=1

                time.sleep(1)

                scroll_height = driver.execute_script("return document.body.scrollHeight;")
                
                if (screen_height * i) > scroll_height:
                    temp = driver.page_source
                    driver.close()
                    break      
                    
            except Exception as e:
                print("TIMED OUT")
                temp = driver.page_source
                driver.close()
                break
                 

        #parse through html code to scrape product urls
        soup = BeautifulSoup(temp, "html.parser")
        for item in soup.find_all(class_ ="Flex__Box-ych44r-1 Grid2__Col-mpt2p4-0 jhfizC"):
            a_tag = item.find("a", class_="Text__LinkText-sc-1e98qiv-0-a Link__StyledAnchor-dkjuk2-0 fiIUU Link__StyledPlainLink-dkjuk2-3 beSDvJ")
            base_url = "https://www.mercari.com"
            link = a_tag.attrs["href"]
            url = urljoin(base_url, link)
            product_urls.append(url)
            
        print("Going to the next url...so far...")
        print(len(product_urls))
    print("scrapping now....Total url is...")
    print(len(product_urls))
    
    return product_urls

In [44]:
def scrape_data(list_of_urls):
    product_df = pd.DataFrame()
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'}
    
    #open a csv file to read in
    with open('Mercarie_data5.csv', 'w', newline='') as csvfile:
        fieldnames = ["title",'price', 'shipping_price',"original_price","number_of_images","number_of_likes","sold_status","condition",
                     "brand","category","tags","posted_date","platform","item_title","url","description","location","user_id",
                     "number_of_stars","number_of_user_reviews","number_of_listed_items","number_of_sold_items","profile_verified",
                     "badges","responsive"]

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for url in list_of_urls:
            product_table = {}
            response = requests.get(url, headers = headers)
            text = BeautifulSoup(response.text, "html.parser")
                        
            if text.find("p", attrs = {"class":"Text-uqn6ov-0 Text__H0-uqn6ov-1 ItemInfo__Heading-ijvfho-2 enHgUe"}):
                
                #title
                if text.find("p", attrs = {"class":"Text-uqn6ov-0 Text__H0-uqn6ov-1 ItemInfo__Heading-ijvfho-2 enHgUe"}):
                    title = text.find("p", attrs = {"class":"Text-uqn6ov-0 Text__H0-uqn6ov-1 ItemInfo__Heading-ijvfho-2 enHgUe"}).string
                else: 
                    print(f"this url has None type for title....: {url} ")

                #price
                if text.find("p", attrs= {"class":"Text-uqn6ov-0 Text__H0-uqn6ov-1 bfDrkK"}):
                    price = text.find("p", attrs= {"class":"Text-uqn6ov-0 Text__H0-uqn6ov-1 bfDrkK"}).string
                else:
                    print(f"this url has None type for price....: {url} ")

                #shipping price 
                if text.find("span", attrs={"class":"LabelChip-bmbtei-0 LabelChip__GreyLabelChip-bmbtei-2 hJnniU"}):
                    shipping_price  = text.find("span", attrs={"class":"LabelChip-bmbtei-0 LabelChip__GreyLabelChip-bmbtei-2 hJnniU"}).string
                else: 
                    shipping_price = None

                #original_price 
                if text.find("p", attrs= {"class":"Text-uqn6ov-0 Text__T1-uqn6ov-9 PriceDrop__ProductOriginalPriceV2-sc-1t3lx6s-0 iDxAof"}):
                    original_price = text.find("p", attrs= {"class":"Text-uqn6ov-0 Text__T1-uqn6ov-9 PriceDrop__ProductOriginalPriceV2-sc-1t3lx6s-0 iDxAof"}).string
                else:
                    original_price = None

                #number of images
                if text.find("div", attrs={"class":"Flex-ych44r-0 Space-cutht5-0 Container-sc-9aa7mx-0 PhotoIndicatorsRevamp__FilmStripWrapper-sc-97b0h4-7 PhotoIndicatorsRevamp__FilmStripWrapperAdapt-sc-97b0h4-12 jZnHGO"}):
                    number_of_images = len(text.find("div", attrs={"class":"Flex-ych44r-0 Space-cutht5-0 Container-sc-9aa7mx-0 PhotoIndicatorsRevamp__FilmStripWrapper-sc-97b0h4-7 PhotoIndicatorsRevamp__FilmStripWrapperAdapt-sc-97b0h4-12 jZnHGO"}).find_all("img"))
                else:
                    number_of_images = None

                #number of likes(hearts)
                if text.find("p", attrs={"class":"Text-uqn6ov-0 Text__H6-uqn6ov-7 iRfdRq"}):
                    number_of_likes = text.find("p", attrs={"class":"Text-uqn6ov-0 Text__H6-uqn6ov-7 iRfdRq"}).string
                else: 
                    number_of_likes = None

                #sold status
                if text.find("button", attrs={"class":"Button__BaseButton-xht50r-0 kmYsKZ"}):
                    sold_status = text.find("button", attrs={"class":"Button__BaseButton-xht50r-0 kmYsKZ"}).string
                else:
                    sold_status = None

                #condition 
                condition = text.find("p", attrs={"class":"Text-uqn6ov-0 Text__T2-uqn6ov-10 dtbBac"}).string

                #Description 
                description = text.find("p", attrs={"class":"Text-uqn6ov-0 Text__T2-uqn6ov-10 Spec__Description-sc-1oxis5o-2 eNOzrJ"}).string

                #Location
                location = [i.text for i in text.find_all("span", attrs={"class":"Spec__ShippingText-sc-1oxis5o-5 cvsNrL"})]

                #User_id
                user_id = text.find("p", attrs={"class":"Text-uqn6ov-0 Text__H2-uqn6ov-3 gMLRcG"}).text

                #number of stars
                if text.find("div", attrs={"class":"Flex-ych44r-0 Stars__StarWrapper-r7vmij-4 eQlgmc"}):
                    number_of_stars = len(text.find("div", attrs={"class":"Flex-ych44r-0 Stars__StarWrapper-r7vmij-4 eQlgmc"}).find_all("svg"))
                else: 
                    number_of_stars = None
                    
                #number of reviews
                number_of_user_reviews = text.find_all("p", attrs={"class":"Text-uqn6ov-0 Text__H6-uqn6ov-7 dAQVwq"})[0].string

                #number of listed items
                number_of_listed_items = text.find_all("p", attrs={"class":"Text-uqn6ov-0 Text__H6-uqn6ov-7 dAQVwq"})[1].string

                #number of sold items
                number_of_sold_items = text.find_all("p", attrs={"class":"Text-uqn6ov-0 Text__H6-uqn6ov-7 dAQVwq"})[2].string

                #responsiveness
                if text.find("p", attrs={"class":"Text-uqn6ov-0 Text__T4-uqn6ov-12 lmKlWK"}): 
                    responsive = text.find("p", attrs={"class":"Text-uqn6ov-0 Text__T4-uqn6ov-12 lmKlWK"}).string 
                else: 
                    responsive = None

                #profile verified 
                if text.find_all("p", attrs={"class":"Text-uqn6ov-0 Text__T3-uqn6ov-11 eJmcXc"}):
                    profile_verified = [i.string for i in text.find_all("p", attrs={"class":"Text-uqn6ov-0 Text__T3-uqn6ov-11 eJmcXc"})]
                else: 
                    profile_verified = None


                #badges
                if text.find_all("img", attrs={"class":"Image-sc-172fqpb-1 SellerBadges__BadgeImage-sc-1vxf73t-2 lgbpFr"}):
                    badges = [i["src"] for i in text.find_all("img", attrs={"class":"Image-sc-172fqpb-1 SellerBadges__BadgeImage-sc-1vxf73t-2 lgbpFr"})]

                else: 
                    badges = None


                #overview section 
                length_category = len(text.find_all("div", attrs={"class":"Flex-ych44r-0 xPWd"}))

                Brand, Tags, Posted_date, Platform, Category, Item_title = None, None, None, None, None, None
                for i in range(length_category):

                    category = text.find_all("div", attrs={"class":"Flex-ych44r-0 xPWd"})[i].find_all("p")[0].string
                    if category == "Brand":
                        Brand = text.find_all("div", attrs={"class":"Flex-ych44r-0 xPWd"})[i].find_all("p")[1].string

                    elif category == "Category":
                        Category = [i.string for i in text.find_all("div", attrs={"class":"Flex-ych44r-0 xPWd"})[i].find_all("a")]

                    elif category == "Tags":
                        Tags = [i.string for i in text.find_all("div", attrs={"class":"Flex-ych44r-0 xPWd"})[i].find_all("a")]

                    elif category == "Posted":
                        Posted_date = text.find_all("div", attrs={"class":"Flex-ych44r-0 xPWd"})[i].find_all("p")[1].string

                    elif category == 'Platform': 
                        Platform  = text.find_all("div", attrs={"class":"Flex-ych44r-0 xPWd"})[i].find_all("p")[1].string

                    elif category == "Item Title":
                        Item_title = text.find_all("div", attrs={"class":"Flex-ych44r-0 xPWd"})[i].find_all("p")[1].string


                product_table = { 
                    "title": title, 
                    "price":price,
                    "shipping_price":shipping_price,
                    "original_price":original_price, 
                    "number_of_images":number_of_images,
                    "number_of_likes":number_of_likes,
                    "sold_status":sold_status,
                    "condition":condition,
                    "brand":Brand,
                    "category":Category,
                    "tags":Tags,
                    "posted_date":Posted_date,
                    "platform":Platform,
                    "item_title":Item_title,
                    "url":url,
                    "description":description,
                    "location":location,
                    "user_id":user_id,
                    "number_of_stars":number_of_stars,
                    "number_of_user_reviews":number_of_user_reviews,
                    "number_of_listed_items":number_of_listed_items,
                    "number_of_sold_items":number_of_sold_items,
                    "profile_verified":profile_verified,
                    "badges":badges,
                    "responsive":responsive
                }

                writer.writerow(product_table)
            
            else: 
                continue 

                
    return product_df


        

In [11]:
def get_user_url(product_urls):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'}
    list_user_urls = []
    global_user_base = {}
    #Scrapping for User profile URLs
    for product_url in product_urls:
        response = requests.get(product_url, headers=headers)
        text = BeautifulSoup(response.text, "html.parser")

        #check if there is a price for the product, if not, continue to the next URL
        if text.find("p", attrs= {"class":"Text-uqn6ov-0 Text__H0-uqn6ov-1 bfDrkK"}) ==None: 
            continue
        else:
            #check if this is a new user, if so continue. if not, skip.
            if text.find("p", attrs={"class":"Text-uqn6ov-0 Text__H2-uqn6ov-3 gMLRcG"}):
                user_id = text.find("p", attrs={"class":"Text-uqn6ov-0 Text__H2-uqn6ov-3 gMLRcG"}).text
            if user_id in global_user_base: 
                continue
            else:
                #find the user link associated with the product 
                if text.find("a", attrs = {"class":"Link__RawAnchor-dkjuk2-2 gvHGid Button__BaseButton-xht50r-0 bgXKXK"}):


                    href = text.find("a", attrs = {"class":"Link__RawAnchor-dkjuk2-2 gvHGid Button__BaseButton-xht50r-0 bgXKXK"})["href"]

                    #once we have profile URLs, we can grab user-specific URLs which we will feed into
                    #scrapping_data function to scrape for their features

                    page = 1
                    global_user_base[user_id] = None

                    while True: 
                        print(user_id)
                        user_url = f"https://www.mercari.com{href}?page={page}"
                        driver = webdriver.Chrome()
                        driver.get(user_url)

                        time.sleep(1)

                        #check if no items were found - if so, break and close chrome driver

                        result = re.findall('/us/item/m\d+', driver.page_source)

                        #if there is nothing to begin with we can break and close 
                        if result == [] or result is None:
                            driver.close()
                            break

                        #if there are less than 42 products on user's profile, no need to look at next page 
                        elif len(result)<42: 
                            driver.close()
                            list_user_urls += [f"https://www.mercari.com{i}" for i in result]
                            break

                        else: 
                            list_user_urls += [f"https://www.mercari.com{i}" for i in result] 

                        driver.close()
                        page+=1

    print("execution done for user urls..going onto next")

    return list_user_urls

In [34]:
# list_of_domain_url = ["https://www.mercari.com/search/?brandIds=319-9318-4578-115-3370-7199-112-54-55-56-8924-7485-10954-2506-4591-3908-5937-7190-115-3370-7199-6027-7019-2766-5016-1805-1811-7224-1816-1806-6848-1808-3752-4239-5609-1400-5074-3153-3909-6771-6483-1099-10954-2859-6027-7189-7192-586-4254-5504-7091-220-233-7363-298-404-548-561-7641-19675-946-23753-17-1076-1161-23751-1239-23757-1442-6732-1480-5058-1566-8072-1611-1686-2176-2239-7194-2425-2462-2537-2562-2618-2949-2798-13816-8722-2993-3107-8867-24909-3476-23830-3778-9263-3929-7188-3976-4086-4127-15094-4554-4565-4712-16037-22435-4878-1296-5089-5299-10222-10261-23796-25117-6036-6716-6683-17680-6241-6334-6449-6472-6480-18354-6625-6647-18413-10792",
#                       "https://www.mercari.com/search/?brandIds=319-9318-4578-115-3370-7199-112-54-55-56-8924-7485-10954-2506-4591-3908-5937-7190-115-3370-7199-6027-7019-2766-5016-1805-1811-7224-1816-1806-6848-1808-3752-4239-5609-1400-5074-3153-3909-6771-6483-1099-10954-2859-6027-7189-7192-586-4254-5504&itemStatuses=2&minPrice=3500",
#                       "https://www.mercari.com/search/?deliveryType=local",
#                       "https://www.mercari.com/search/?categoryIds=67&minPrice=1500&sortBy=1"
#                       "https://www.mercari.com/search/?categoryIds=1594-82-81-1595&minPrice=6000&sortBy=1"
#                       "https://www.mercari.com/search/?categoryIds=579&minPrice=1500&sortBy=1",
#                       "https://www.mercari.com/search/?keyword=Athletic%20apparel&minPrice=1500&sortBy=1"
#                       "https://www.mercari.com/search/?categoryIds=559-1950-28-263&minPrice=2000&sortBy=1",
#                       "https://www.mercari.com/search/?authenticity=lux&categoryIds=21&sortBy=1",
#                       "https://www.mercari.com/search/?categoryIds=585&itemConditions=1-2&maxPrice=15000&minPrice=1000&sortBy=1",
#                       "https://www.mercari.com/search/?categoryIds=69&itemConditions=1-2&maxPrice=5000&minPrice=1000&sortBy=1",
#                       "https://www.mercari.com/search/?categoryIds=1713&itemConditions=1-2&maxPrice=5000&minPrice=1000&sortBy=1",
#                       "https://www.mercari.com/search/?authenticity=lux"
#                     ]


In [46]:
df = scrape_data(get_product_urls())

TIMED OUT
Going to the next url...so far...
530
Going to the next url...so far...
860
TIMED OUT
Going to the next url...so far...
1390
TIMED OUT
Going to the next url...so far...
1920
TIMED OUT
Going to the next url...so far...
2450
TIMED OUT
Going to the next url...so far...
2978
TIMED OUT
Going to the next url...so far...
3508
TIMED OUT
Going to the next url...so far...
4037
Going to the next url...so far...
4367
TIMED OUT
Going to the next url...so far...
4897
TIMED OUT
Going to the next url...so far...
5426
TIMED OUT
Going to the next url...so far...
5955
TIMED OUT
Going to the next url...so far...
6485
TIMED OUT
Going to the next url...so far...
7015
TIMED OUT
Going to the next url...so far...
7545
TIMED OUT
Going to the next url...so far...
8072
Going to the next url...so far...
8302
TIMED OUT
Going to the next url...so far...
8823
TIMED OUT
Going to the next url...so far...
9351
TIMED OUT
Going to the next url...so far...
9871
TIMED OUT
Going to the next url...so far...
10399
TI